In [7]:
import torch
from transformer import Transformer
import os
import random
import pandas as pd

N=30
dim = 120
l=1
h=1
f=128

iters = [1000*i for i in range(1, 10)]
#iters = [1000*i for i in range(1, 65)]

folderName = f"{N}_{dim}_{l}_{h}_{f}"

model = Transformer(N, dim, h, l, f, 1e-5).cuda()

def validate(model, func, num_samples=1000):
      inputs = torch.tensor([random.randint(0, 2**N-1) for _ in range(num_samples)]).cuda()
      targets = torch.FloatTensor([float(func(x)) for x in inputs]).cuda()
      result = model(inputs).cuda()
      #print("result[0]: " + str(result[0]) + ", target[0]: " + str(targets[0]))
      diffs = result - targets
      #print("(result-targets): " + str(diffs[0]))
      losses = diffs.pow(2)
      #print("(result-targets).opw(2): " + str(losses[0]))
      loss = (result - targets).pow(2).mean()
      return loss.detach()

def validate_training(model, func, dataloader):
      total_size=0
      total_loss = 0
      for idx, inputs in enumerate(train_dataloader):
        targets = torch.FloatTensor([float(func(x)) for x in inputs]).cuda()
        result = model(inputs).cuda()
        #print("result shape: "+ str(result.shape) + ", target shape: " + str(targets.shape))
          
        total_loss += (result - targets).pow(2).sum().detach()
        #print("total_loss: " + str(total_loss))
        total_size += inputs.shape[0]
          
      print("total_loss: " + str(total_loss) + ", total size: " + str(total_size))
      loss = float(total_loss/total_size)
      
      return loss
    
def rboolf(N, combs,coefficients):

    def func(x):
        binary = f"{x:0{N}b}"+"0"
        comps = []
        for elem in combs:
            res = 1
            for e in elem:
                bit = 1 if int(binary[e]) else -1
                res *= bit
            comps.append(res)
        return torch.dot(coefficients, torch.tensor(comps, dtype=torch.float32).cuda())
    return func, (coefficients, combs)



for width in range(1, N, 3):
    for deg in range(2,5):    
        for i in range(5):
            for iter in iters:
                dir_name = os.path.join(folderName, f"func{i}_deg{deg}_width{width}")
                model = Transformer(N, dim,h,l,f,1e-5).cuda()
                # Load the state dictionary from a file
                #try:
                print("trying to load " + str(dir_name+"/model_"+str(iter)+".pt"))
                state_dict = torch.load("/dartfs/rc/lab/C/CybenkoG/bool_sens/" +dir_name+"/model_"+str(iter)+".pt")
                summary=pd.read_csv(dir_name+ "/curr_func.csv") 
                last_line = summary.loc[len(summary)-1]
                print("last line of summary: " +str(summary))
                print("loaded state dict" ) 
                # Load the state dictionary into the model
                model.load_state_dict(state_dict)
                # Get Test Loss
                model.eval()
                combs = torch.load(f"{dir_name}/func_combs.pt")
                coeffs = torch.load(f"{dir_name}/func_coeffs.pt")
                func,_ = rboolf(N,combs,coeffs)
                test_loss = float(validate(model=model, func=func, num_samples=1000).cpu())
                train_dataloader = torch.load(f"{dir_name}/train_dataloader.pt")
                train_loss = float(validate_training(model=model,func=func,dataloader = train_dataloader))
                 
                summary_csv = f"{folderName}/test_summary_new3.csv"
                func_summary = pd.DataFrame(columns=["train_iters", "test_loss","train_loss","width","deg","func"])
                func_summary.loc[len(func_summary)] = {"train_iters":iter, "test_loss":test_loss,"train_loss":train_loss,"deg":deg,"width":width,"func":i}
                print("iter: " + str(iter) + ", func: " + str(i) + ", deg: " + str(deg) + ", width: " + str(width)+", test loss: " + str(test_loss)+", train loss: " + str(train_loss))
                func_summary.to_csv(summary_csv, mode='a', header=not os.path.exists(summary_csv), index=False)
                # except Exception as e:
                #     # Code to execute for any other exception
                #     print("An error occurred:", e)
                #     print("iter: " + str(iter) + ", func: " + str(i) + ", deg: " + str(deg) + ", width: " + str(width))
                #     print("no file found for this combo")


trying to load 30_120_1_1_128/func0_deg2_width1/model_1000.pt
last line of summary:      Unnamed: 0   iter      loss
0             0    790  0.018193
1             1    800  0.035537
2             2    810  0.049525
3             3    820  0.062341
4             4    830  0.074527
..          ...    ...       ...
412         412  56050  0.007035
413         413  57620  0.006959
414         414  59190  0.006863
415         415  60760  0.006815
416         416  62330  0.006679

[417 rows x 3 columns]
loaded state dict
total_loss: tensor(94233.3438, device='cuda:0'), total size: 10000
iter: 1000, func: 0, deg: 2, width: 1, test loss: 0.1551576554775238, train loss: 9.423334121704102
trying to load 30_120_1_1_128/func0_deg2_width1/model_2000.pt
last line of summary:      Unnamed: 0   iter      loss
0             0    790  0.018193
1             1    800  0.035537
2             2    810  0.049525
3             3    820  0.062341
4             4    830  0.074527
..          ...    ...       

KeyboardInterrupt: 

In [ ]:
import torch
from transformer import Transformer
import os
import random
import pandas as pd

dir_name = "30_120_1_1_128/func0_deg2_width1"

iter=5000
state_dict = torch.load( dir_name+"/model_"+str(iter)+".pt")


In [2]:
dir_name+"/model_"+str(iter)+".pt"

'30_120_1_1_128/func0_deg2_width1/model_5000.pt'